# 공휴일 정보

In [ ]:
import requests
from urllib import parse

url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/"
api_key_utf8 = "yDvCX02B8DyjpXc9BVBID6jJqzO5Io2nDD9Bv8QCcNtiRfCwgtQriPAs3KOQPpWICSM%2BHzzNPdVQXLjWJpfm5w%3D%3D"
api_key_decode = parse.unquote(api_key_utf8)

url_holiday = url + "getRestDeInfo"
params = {
    "ServiceKey": api_key_decode,
    "solYear": 2021,
    "numOfRows": 100
}

response = requests.get(url_holiday, params=params)

In [ ]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dateKind>01</dateKind><dateName>1월1일</dateName><isHoliday>Y</isHoliday><locdate>20210101</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>설날</dateName><isHoliday>Y</isHoliday><locdate>20210211</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>설날</dateName><isHoliday>Y</isHoliday><locdate>20210212</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>설날</dateName><isHoliday>Y</isHoliday><locdate>20210213</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>삼일절</dateName><isHoliday>Y</isHoliday><locdate>20210301</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>어린이날</dateName><isHoliday>Y</isHoliday><locdate>20210505</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>부처님오신날</dateName><isHoliday>Y</isHoliday><locdate>20210519</loc

In [ ]:
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def getHoliday(year: int) -> pd.DataFrame:
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"
    api_key_utf8 = "yDvCX02B8DyjpXc9BVBID6jJqzO5Io2nDD9Bv8QCcNtiRfCwgtQriPAs3KOQPpWICSM%2BHzzNPdVQXLjWJpfm5w%3D%3D"
    api_key_decode = parse.unquote(api_key_utf8)

    params = {
        "ServiceKey": api_key_decode,
        "solYear": year,
        "numOfRows": 100
    }
    response = requests.get(url, params=params)
    xml = BeautifulSoup(response.text, "lxml")
    items = xml.find('items')
    item_list = []
    for item in items:
        dt = datetime.strptime(item.find("locdate").text.strip(), '%Y%m%d')
        item_dict = {
            "이름": item.find("datename").text.strip(),
            "날짜": dt,
            "요일": dt.weekday()
        }
        item_list.append(item_dict)

    return pd.DataFrame(item_list)

In [ ]:
def getAnniversaryInfo(year: int) -> pd.DataFrame:
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo"
    api_key_utf8 = "yDvCX02B8DyjpXc9BVBID6jJqzO5Io2nDD9Bv8QCcNtiRfCwgtQriPAs3KOQPpWICSM%2BHzzNPdVQXLjWJpfm5w%3D%3D"
    api_key_decode = parse.unquote(api_key_utf8)

    params = {
        "ServiceKey": api_key_decode,
        "solYear": year,
        "numOfRows": 100
    }

    temp = ["0", "1", "2", "3", "4", "5", "6"]

    response = requests.get(url, params=params)
    xml = BeautifulSoup(response.text, "lxml")
    items = xml.find('items')
    item_list = []
    for item in items:
        dt = datetime.strptime(item.find("locdate").text.strip(), '%Y%m%d')
        item_dict = {
            "이름": item.find("datename").text.strip(),
            "날짜": dt,
            "요일": dt.weekday()
        }
        item_list.append(item_dict)

    return pd.DataFrame(item_list)

In [ ]:
holiday = getHoliday(2021)
holiday.at[0, "이름"] = "신정"
holiday

,이름,날짜,요일
0,신정,2021-01-01,4
1,설날,2021-02-11,3
2,설날,2021-02-12,4
3,설날,2021-02-13,5
4,삼일절,2021-03-01,0
5,어린이날,2021-05-05,2
6,부처님오신날,2021-05-19,2
7,현충일,2021-06-06,6
8,광복절,2021-08-15,6
9,대체공휴일,2021-08-16,0


- 기념일은 의미 없어 보임

In [ ]:
anniversary = getAnniversaryInfo(2021)
anniversary

,이름,날짜,요일
0,2·28 민주운동 기념일,2021-02-28,6
1,납세자의 날,2021-03-03,2
2,3·8민주의거 기념일,2021-03-08,0
3,3·15 의거 기념일,2021-03-15,0
4,상공의 날,2021-03-17,2
...,...,...,...
61,아동학대예방의 날,2021-11-19,4
62,소비자의 날,2021-12-03,4
63,무역의 날,2021-12-05,6
64,자원봉사자의 날,2021-12-05,6


In [ ]:
holiday.to_csv("/drive/MyDrive/LDA/holiday.csv", index = False)